Reference:

http://gym.openai.com/docs/#spaces

https://github.com/openai/gym/wiki

https://www.kaggle.com/code/mpware/game-observations

https://www.kaggle.com/code/gabrielmilan/drl-ga-starter

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%capture
# install the latest version of kaggle_environments
!pip install --upgrade kaggle_environments

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
print(env.name, env.version)

# Specification

In [ ]:
for key in env.specification.keys():
    print(key)
    print(env.specification[key])
    print("\n")

In [ ]:
for key in env.specification['configuration'].keys():
    print("self." + key + " = ks_env.configuration." + key)

In [ ]:
for key in env.specification['configuration'].keys():
    print(key)
    print(env.specification['configuration'][key])
    print("\n")

In [ ]:
for key in env.specification['observation'].keys():
    print(key)
    print(env.specification['observation'][key])
    print("\n")

# Observation Space

e.g. Tuple(Box(1,), Discrete(283), Discrete(283), Discrete(283), Box(1,))

-> spaces.Box(low=0, high=1000?, shape=(self.size,self.size,5), dtype=int)

In [ ]:
observation = env.reset()

## kore

In [ ]:
np.array(observation[0]["observation"]["kore"]).reshape(21,21)

## Player/Opponent fleets

In [ ]:
size = 21

In [ ]:
def get_fleets(player_fleets_):
    player_fleets_grid = np.zeros((size, size), dtype=np.int32)
    for turn, player_fleets_turn in enumerate(player_fleets_):
        for (fleet_id, [fleet_index, fleet_kore, ship_count, direction, flight_plan]) in player_fleets_turn.items():
            (x, y) = Point.from_index(fleet_index, GRID_SIZE)
            fleet_direction = Direction.from_index(direction)        
            player_fleets_grid[y, x] = ship_count
    return player_fleets_grid

In [ ]:
player_fleets_grid = get_fleets(observation[0]["observation"]["players"][0][2])
player_fleets_grid

In [ ]:
# opponent_fleets_grid = get_fleets(observation[0]["observation"]["players"][1][2])
# opponent_fleets_grid

## Player/Opponent shipyards

In [ ]:
def get_name_player(actions_):
    player = {}
    for action_ in actions_:
        names_player = list(action_.keys())
        for name_player in names_player:
            if player.get(name_player) is None:
                player[name_player] = 1
            else:
                player[name_player] = player[name_player] + 1
    return player

def get_player_shipyard(player_shipyards_):
    player_shipyards_dict_ = {}
    player_shipyards_grid = np.zeros((size, size), dtype=np.int32)
    # For each shipyard
    for player_init in list(get_name_player(player_shipyards_).keys()):
        shipyard_pd = []
        # For each turn, get shipyard
        for turn, player_shipyard_ in enumerate(player_shipyards_, 0):
            shipyard_ = player_shipyard_.get(player_init)
            # If shipyard available in the current turn
            if shipyard_ is not None:
                shipyard_index, ship_count, turns_controlled = shipyard_
                shipyard_pd.append((player_init, turn, shipyard_index, ship_count, turns_controlled))
        shipyards_pd = pd.DataFrame(shipyard_pd, columns=["name_player", "turn", "location", "ship_count", "turns_controlled"])
        player_shipyards_dict_[player_init] = shipyards_pd
        for turn in range(len(player_shipyards_)):
            turn_pd = shipyards_pd[shipyards_pd["turn"] == turn]
            if len(turn_pd) > 0:
                shipyard_index, shipyard_ships = turn_pd[["location", "ship_count"]].values[0]
                (x, y) = Point.from_index(shipyard_index, size)
                player_shipyards_grid[y, x] = shipyard_ships + 1 # +1 to make it exist
    return player_shipyards_grid

In [ ]:
player_shipyards_grid = get_player_shipyard(observation[0]["observation"]["players"][0][2])
player_shipyards_grid

In [ ]:
# opponent_shipyards_grid = get_player_shipyard(observation[0]["observation"]["players"][1][2])
# opponent_shipyards_grid

# Action space

In [ ]:
from kaggle_environments.envs.kore_fleets.helpers import *

In [ ]:
help(ShipyardAction)

An example of action space

Discrete(6)

| Num  | Action | 
| ---- | ----   |
| 0    | Do nothing |
| 1    | Spawn one ship | 
| 2    | Launch a fleet north |
| 3    | Launch a fleet east |
| 4    | Launch a fleet west |
| 5    | Launch a fleet south |